# Connect to the googledrive with PDF Files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Assign a default path for the pdfs

In [0]:
#default path for the folder inside google drive
default_path = '/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/data/pdf_to_text'#path for training text (article)
#train_article_path = default_path + "/train/train.article.txt"  #path for training text output (headline)


# Install the Packages Required


In [0]:
!pip install PyPDF2 
!pip install textract
!pip install nltk
!pip install eazymind
!pip install bert-extractive-summarizer
!pip install spacy==2.1.3
!pip install transformers
!pip install pyrouge


     |████████████████████████████████| 81kB 4.6MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61085 sha256=ae5f27ac2b570be31dc6d6bd0f06dea9e07e8d096c36d8973689b08ec774df77
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
     |████████████████████████████████| 112kB 9.7MB/s 
     |████████████████████████████████| 5.6MB 17.1MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 8.9MB 45.6MB/s 
     |████████████████████████████████| 32.8MB 36.4MB/s 
     |████████████████████████████████| 112kB 48.5MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 9.7MB 43.8MB/s 
     |████████████████████████████████| 143kB 37.7MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
  Created wheel for EbookLib: filename=EbookLib-0.17.1-cp36-none-any.whl size=38164 sha256=1561ff9ac59e1

In [0]:
import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!java -version

openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


# Test Cuda


In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


In [0]:
%reload_ext nvcc_plugin


directory /content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/src already exists
Out bin /content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/result.out


In [0]:
%%cu
#include <iostream>
int main() {
    std::cout << "Hello world\n";
    return 0;
}

Hello world



In [0]:
%%cu
#include <stdio.h>
#define N  64
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
__global__ void matrixMulGPU( int * a, int * b, int * c )
{
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] * b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}
/*
 * This CPU function already works, and will run to create a solution matrix
 * against which to verify your work building out the matrixMulGPU kernel.
 */
void matrixMulCPU( int * a, int * b, int * c )
{
  int val = 0;
for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      val = 0;
      for ( int k = 0; k < N; ++k )
        val += a[row * N + k] * b[k * N + col];
      c[row * N + col] = val;
    }
}
int main()
{
  int *a, *b, *c_cpu, *c_gpu; // Allocate a solution matrix for both the CPU and the GPU operations
int size = N * N * sizeof (int); // Number of bytes of an N x N matrix
// Allocate memory
  cudaMallocManaged (&a, size);
  cudaMallocManaged (&b, size);
  cudaMallocManaged (&c_cpu, size);
  cudaMallocManaged (&c_gpu, size);
// Initialize memory; create 2D matrices
  for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      a[row*N + col] = row;
      b[row*N + col] = col+2;
      c_cpu[row*N + col] = 0;
      c_gpu[row*N + col] = 0;
    }
/*
   * Assign `threads_per_block` and `number_of_blocks` 2D values
   * that can be used in matrixMulGPU above.
   */
dim3 threads_per_block(32, 32, 1);
  dim3 number_of_blocks(N / threads_per_block.x + 1, N / threads_per_block.y + 1, 1);
matrixMulGPU <<< number_of_blocks, threads_per_block >>> ( a, b, c_gpu );
checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
checkCudaErr(cudaGetLastError(), "GPU");
// Call the CPU version to check our work
  matrixMulCPU( a, b, c_cpu );
// Compare the two answers to make sure they are equal
  bool error = false;
  for( int row = 0; row < N && !error; ++row )
    for( int col = 0; col < N && !error; ++col )
      if (c_cpu[row * N + col] != c_gpu[row * N + col])
      {
        printf("FOUND ERROR at c[%d][%d]\n", row, col);
        error = true;
        break;
      }
if (!error)
    printf("Success!\n");
// Free all our allocated memory
  cudaFree(a); cudaFree(b);
  cudaFree( c_cpu ); cudaFree( c_gpu );
}

Success!



#  Import Packages

In [0]:
import PyPDF2
import textract
import nltk
import eazymind
import pandas as pd
from nltk.tokenize import sent_tokenize
from summarizer import Summarizer

100%|██████████| 40155833/40155833 [00:02<00:00, 18843365.66B/s]


In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import torch  


In [0]:
torch.cuda.current_device()

0

# Load Bert Summarizer

In [0]:
model = Summarizer()

100%|██████████| 231508/231508 [00:00<00:00, 939850.43B/s]


# Function that takes a pdf file and converts it to text

In [0]:
def pdf_to_text(document):
    try:
        filename = document

        with open(filename,'rb') as f:
            pdfReader = PyPDF2.PdfFileReader(f)
        #pdfFileObj = open(filename, 'rb')
            num_pages = pdfReader.numPages
            count = 0
            text = ''
            while count < num_pages:
                pageObj = pdfReader.getPage(count)
                count += 1
                text += pageObj.extractText()
            if text != '':
                text = text
            else: 
                text = textract.process(f, method = 'tesseract', language = 'eng')
            return [document, text]
    except Exception as e:
        print(e)

## Take a pdf document from the folder and input the name into the function.

In [0]:
%cd /content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/data/pdf_to_text

/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/data/pdf_to_text


In [0]:
pdf = pdf_to_text("PrideAndPrejudice.pdf")

In [0]:
pd.set_option('display.max_colwidth', -1)

In [0]:
df = pd.DataFrame(pdf)

In [0]:
df_transposed = df.T.reset_index().reindex(columns=[0,1,'index'])

In [0]:
df = df_transposed.rename(columns={0: u"Source", 1: u"Text"})

In [0]:
df

In [0]:
df = df.replace('\n','', regex=True)
df = df.replace('˜','', regex=True)
df = df.replace(r"˝|˚|˛|\\\|˙|ˆ|ˇ|˘|/|:|\*|%|\+|<|>|\(|\)|\\|[|]", '', regex=True)
df = df.replace(r'˙{1:}', '', regex=True)
df = df.replace(r'\.\s{1,}', '', regex=True)
df = df.replace(r'\,\s{1,}', '', regex=True)
df = df.replace(r'-{1,}', '', regex=True)
df = df.replace(r'\.{2,}', '', regex=True)
df = df.replace(r'\,{2,}', '', regex=True)
#df = df.replace(r'\w{2,}\d\w', '', regex=True)
df = df.replace(r'\w{14,}', '', regex=True)
#df = df.replace('(?:\.\w+)*', '', regex=True)
df = df.replace(r"\.", '. ', regex=True )
df = df.replace(r'ﬁ|ﬂ|™|˙˙˙˙|Ł|Œ|˙˙|˙|–', '', regex=True)
df = df.replace(r'\d+', '', regex=True)
#df = df.replace(r"\b\d+\b", '', regex=True)
df = df.replace(r"\s\s+", " ", regex=True)
df = df.replace(r'\s\.', '.', regex=True)



In [0]:
doc_text = df['Text'].to_list()

In [0]:
# using list comprehension to compress the list to a sting 
listToStr = ' '.join(map(str, doc_text))

In [0]:
len(listToStr)

479761

In [0]:
tok_sentences = sent_tokenize(listToStr)

In [0]:
tok_sentences

In [0]:
len(tok_sentences)

381

In [0]:
model1 = "this , this , would and him . mrs bennet , mrs , mrs and mrs . the ! i . mr manners ! . the likewise likewise of us . compliments on the compliments . what to do again for ? . the valuable of lady catherine . the atonement of the atonement . relief elizabeth young will , young , dies .` jane ' town , ! , ! the agreeable quality of agreeable quality . uncle and aunt , uncle , proposed and aunt . hard upon younger sisters . the lastborn motive of youth . ? in , ? , lady dances . civility at civility . the will of the will . brighton , us of soldiers and soldiers . ! ! laughed . manners manners , gentleness , dies . ! ! , ! , dies . the neglect of girl 's girl . the charge of ruled . ! eastbourne . sister gardiner my , sister . the bell of the bell . hi de bourgh de bourgh . a marriage of marriage . mr darcy is , mr , dies ."

In [0]:
len(model1)

833

In [0]:
new_text = [ element for element in tok_sentences if len(element) >= 110 ]

In [0]:
#using list comprehension to compress the list back to a sting 
doc_of_sent = ' '.join(map(str, new_text))

In [0]:
doc_of_sent

'Title Pride and Prejudice Chapter It is a truth universally a single man in possession of a good fortunemust be in want of a wifeHowever little known the feelings or views of such a man may be on his first entering a truth is so well fixed in the minds of the surrounding familiesthat he is considered the rightful property of some one or other of their replied that he had notMrBennet made no was invitation enoughMrsLong says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the placeand was so much delighted with itthat he agreed with MrMorris immediately; that he is to take possession before Michaelmasand some of his servants are to be in the house by inglemy dearto be sure! You must know that I am thinking of his marrying one of that he _may_ fall in love with one of themand therefore you send them by perhaps will be still betterfor as you are as handsome as any of themMrBingley may like you be

# Pass the doc_of_sent to BertSummarizer to create an extractive summarization

In [0]:
result = model(doc_of_sent, min_length=100)
full = ''.join(result)

In [0]:
len(full)

4832

In [0]:
print(full)

If I had known as much this morning I certainly would not have called on himIt is very unlucky; but as I have actually The astonishment of the ladies was just what he wished; that of MrsBennet perhaps surpassing the rest; thoughwhen the first tumult of joy was overshe began to declare that it was what she had expected all the whilepersuade you at lastI was sure you loved your girls too well to neglect such an pleased I am! I wish you had been theremy dearto have given him one of your setdownsI quite Chapter When Jane and Elizabeth were alonethe formerwho had been cautious in her praise of MrBingley to her sister just how very much she admired ; and I never saw such happy manners!so ought likewise to beif he possibly canHis character is eat difference between usCompliments always take _you_ by surpriseand _me_ neverWhat could be more natural than his asking you again? I remaindear sirwith respectful compliments to your lady and daughtersyour wellwisher and friendmaking seems to be a mos

In [0]:
with open("prideandpred.txt", "w") as text_file:
    text_file.write(full)

In [0]:
pwd

'/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/data/pdf_to_text'

In [0]:
len(full), len(doc_of_sent)

(4732, 473779)

In [0]:
tok_sentences = sent_tokenize(full)
new_text = [ element for element in tok_sentences if len(element) >= 110 ]

In [0]:
len(tok_sentences), len(new_text)

(15, 15)

In [0]:
new_text

['If I had known as much this morning I certainly would not have called on himIt is very unlucky; but as I have actually The astonishment of the ladies was just what he wished; that of MrsBennet perhaps surpassing the rest; thoughwhen the first tumult of joy was overshe began to declare that it was what she had expected all the whilepersuade you at lastI was sure you loved your girls too well to neglect such an pleased I am!',
 'I wish you had been theremy dearto have given him one of your setdownsI quite Chapter When Jane and Elizabeth were alonethe formerwho had been cautious in her praise of MrBingley to her sister just how very much she admired ; and I never saw such happy manners!so ought likewise to beif he possibly canHis character is eat difference between usCompliments always take _you_ by surpriseand _me_ neverWhat could be more natural than his asking you again?',
 'I remaindear sirwith respectful compliments to your lady and daughtersyour wellwisher and friendmaking seems t

# Load easymind Summarizer

In [0]:
from eazymind.nlp.eazysum import Summarizer
key = "d46a5c16d4440c12907c7aaa5eac002a"
summarizer = Summarizer(key)

# Pass each item in the list of string to the eazymind summarizer

In [0]:
for sentence in new :
  print(summarizer.run(new_text[1]))

In [0]:
sample = "I wish you had been there my dear to have given him one of your set downs I quite Chapter When Jane and Elizabeth were alone the former who had been cautious in her praise of Mr Bingley to her sister just how very much she admired ; and I never saw such happy manners! So ought likewise to be if he possibly can. His character is eat difference between us. Compliments always take you by surprise and me never. What could be more natural than his asking you again?"

In [0]:
summarizer.run(sample)

'jane and elizabeth were alone the former who had been cautious in her praise of mr bingley to her sister just how very much she admired admired to have given him one of your set downs .'

# Load Joint_Parse_Summ

In [0]:
%cd '/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ'
#https://github.com/ucfnlp/joint-parse-n-summarize
#https://github.com/dsandeep0138/Deep-learning-Andrew-NG-Coursera/tree/master/Convolutional%20Neural%20Networks

/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ


In [0]:
#Do not need to download every time. 
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
!unzip stanford-corenlp-full-2018-10-05.zip


In [0]:
%cd stanford-corenlp-full-2018-10-05

/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/stanford-corenlp-full-2018-10-05


In [0]:
pwd

'/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ/stanford-corenlp-full-2018-10-05'

In [0]:
LOGDIR = '/log'
get_ipython().system_raw(
    'java -mx8g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9900 -timeout 60000 &'
    .format(LOGDIR)
)

#java -mx8g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000
#https://stackoverflow.com/questions/56001048/how-to-run-stanford-corenlp-server-on-google-colab

In [0]:
# Use ngrok to tunnel traffic to localhost
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


--2019-12-06 03:20:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.235.200.97, 54.165.145.59, 34.227.147.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.235.200.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.5’

ngrok-stable-linux- 100%[===================>]  13.13M  32.7MB/s    in 0.4s    

2019-12-06 03:20:46 (32.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.5’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 9900 &')

In [0]:
# Retrieve public url
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://5ffbce15.ngrok.io


In [0]:
%cd /content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ

/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ


In [0]:
pwd

'/content/drive/Shared drives/IST 664 NLP/Abstract/joint_parse_summ'

In [0]:
!python run.py --do_test --inputFile data/pdf_to_text/prideandpred.txt

Namespace(data='gigaword', do_test=True, do_train=False, inputFile='data/pdf_to_text/prideandpred.txt', model='deRNNG', optionsFrame={'dataset': 'settings/dataset/gigaword.json', 'vocabulary': 'settings/vocab/gigaword.json', 'network': 'settings/network/deRNNG_gigaword.json', 'test': 'settings/test/test.json'}, train_prefix='train', valid_prefix='valid')
CUDA 0
         0.2627 : Loading Options Frames
         0.2652 : dataset options are loading 
         0.8883 : vocabulary options are loading 
         1.5686 : network options are loading 
         2.2012 : test options are loading 
         2.7758 : 
Options:

{
	dataset : 
	{
		name : gigaword
		method : load
		srcFeatList : []
		refFeatList : ['action_map']
		Parts : 
		{
			train : 
			{
				name : train
				path : ../../dataset/gigaword_eng_5/train
				sorted : True
				shuffled : False
			}
			sTrain : 
			{
				name : sTrain
				path : ../../dataset/gigaword_eng_5/sTrain
				sorted : True
				shuffled : False
			}
			mTrain :

In [0]:
!python run.py --do_test --data newsroom --inputFile data/pdf_to_text/cyber.txt

# DeepAi Text Summary

In [0]:
curl \
    -F 'text=YOUR_TEXT_URL' \
    -H 'api-key:6232397c-49e7-459c-9cef-e6133e93dd3b' \
    https://api.deepai.org/api/summarization 

SyntaxError: ignored

In [0]:
# Example directly sending a text string:

curl \
    -F 'text="OPERATION GHOSTThe Dukes arent back Š they never leftMatthieu Faou Mathieu Tartare Thomas DupuyESET Research White papers October TABLE OF summary Background. Table RegDuke Windows registry keysTable Hardcoded FatDuke backdoor commands Table UserAgent strings used by LiteDuke Figure Invalid digital signature added to the backdoor Figure Control flow flattening used to obfuscate the MiniDuke backdoor. TargetsOver the yearsit has been possible to draw the big picture of the Dukes main targetsThe group is primarily interested in spying on governments either in the West or in former USSR group has also targeted various organizations linked to NATOthink tanksand political parties. In CloudDuke the operators leveraged cloud storage services such as OneDrive as their C&C channelsThey were not the rst group to use this techniquebut it is geers as it is harder for defenders to spot hostile connections to legitimate cloud storage services than to other suspicious or lowreputation URLs. We spent months apparently chasing a ghost thena few months agowe were able to attribute several distinct intrusions to the DukesDuring the analysis of those intrusionswe uncovered several new malware families and FatDukeWe call the Dukes campaigns using these newly discovered tools Operation Ghost.. Targets and timelineWe believe Operation Ghost started in and was still ongoing as of this writingOur research shows paignWe also have discovered an inltration by the Dukes at the WashingtonDC embassy of a European Union country. Howeveran attribution based only on the presence of known Dukes tools on the same machines should be taken with a grain of saltWe also found two other APT threat actors Turla and Sednit on some of the same computers. Firstthey are very steal credentials and use them to move laterally on the networkWe have seen them using credentials to compromise or recompromise machines on the same local responding to a Dukes compromiseit is important to make sure to remove every implant in a short period of attackers will use any remaining implant to compromise the cleaned systems again. RegDukea recovery rst stagewhich uses Dropbox as its C&C serverThe main payload is encrypted on disk and the encryption key is stored in the Windows registryIt also relies on steganography as above. FatDukethe third stageThis sophisticated backdoor implements a lot of and has a very ny opaque recompile it and modify the obfuscation frequently to bypass security product detections. For Operation times The time at which they uploaded C&C pictures to the Dropbox account used by RegDuke The time at which they posted encoded C&C URLs on the social media accounts used by PolyglotDuke The compilation timestamps of dozens of samplesWe believe they were not tampered withas they are consistent with what we see in ESET telemetry data. The custom string encryption algorithm used by the PolyglotDuke dropper is identical to the one used by well as other samples from previous Dukes campaignsand is depicted in Figure. The C&C server address is retrieved and decoded from various public webpages such as ImgurImgBB or Fotolog public encrypted public webpage URLs are hardcoded in each sample from three to six URLs in a single sample and it will iterate over the hardcoded list of C&C server addresses until it is able to decode a valid C&C URL example of a public webpage containing an encoded C&C URL is shown Figure. png lead us to think that they were valid images used to look likTo extract the payload from the le downloaded from the C&C will rst decrypt the last eight bytes with RC using the same key as the one used for strings decryptionThe rst four elative to the end of the le and the last four bytes provide a value used as integrity check; that value is the same as the rst four bytes at the beginning of the blob. door functions such as Uploading or downloading les Creating processes Getting system information method Getting the list of local drives and their type Reading and writing in the named pipe Starting and stopping the proxy feature. Firstit can probe the UserAgent by making an HTTP request on a socket it has just createdIt creates a socket listening on accepts any connectionIt calls ShellExecuteW with open and httplocalhost as argumentThis will open the default browser on the URL localhost. OKServer Apache... close; simple JavaScript code will directly close the browserThe window pops up only for a fraction of second but the user also loses focus of the currently active window. exe Query FO {"commandBody"" redactedC$ Example of commands sent to FatDukeThese JSON objects contain a command identier and the command shows the commands implemented by FatDuke. Firstthey use string stacking for all important strings; this consists of building strings dynamically by pushing each character separately on the stackrather than using strings from the. Figure Assembler used by the developer screenshot of DIE analysisThe backdoor DLL exports seven functions that have relatively explicit names CC being Crypto Container SendBin LoadFromCC SaveToCC GetDBHandle GetCCFieldLn malware apparently attempts to bypass Kaspersky security products by checking if the registry key exists and if soit waits seconds before executing any additional codeWe do not know whether this really bypasses any Kaspersky security products. d using the following ;CHAR;The conguration default values are hardcoded in the binaryThis SQLite table allows the malware operators to update these parameters moduleswhich are plugins for the backdoorare stored in the databaseSince the database is encryptedthe modules never touch the disk in plaintext and will only be loaded into have not yet been able to nd any of the plugins used by LiteDuke. This campaign also shows that APT threat actors going dark for several years does not mean they have stopped spyingThey might pause for a while and reappear in another formbut they still need to spy to fulll their mandates."' \
    -H 'api-key:6232397c-49e7-459c-9cef-e6133e93dd3b' \
    https://api.deepai.org/api/summarization 

In [0]:
pwd

'/content'

In [0]:
import requests

headers = {
    'api-key': '6232397c-49e7-459c-9cef-e6133e93dd3b',
}

files = {
    'text': ('/content/Output.txt', open('/content/Output.txt', 'rb')),
}

response = requests.post('https://api.deepai.org/api/summarization', headers=headers, files=files)

In [0]:
response

<Response [200]>

In [0]:
import requests

headers = {
    'api-key': '6232397c-49e7-459c-9cef-e6133e93dd3b',
}

files = {
    'text': (None, full),
}

response = requests.post('https://api.deepai.org/api/summarization', headers=headers, files=files)

In [0]:
response.json()

{'id': '3233b429-4f48-4080-ae5c-0053a6d6b16f',
 'output': 'I wish you had been theremy dearto have given him one of your setdownsI quite Chapter When Jane and Elizabeth were alonethe formerwho had been cautious in her praise of MrBingley to her sister just how very much she admired ; and I never saw such happy manners!so ought likewise to beif he possibly canHis character is eat difference between usCompliments always take _you_ by surpriseand _me_ neverWhat could be more natural than his asking you again?\nI remaindear sirwith respectful compliments to your lady and daughtersyour wellwisher and friendmaking seems to be a most conscientious and polite young manupon my wordand I doubt not will prove a valuable if Lady Catherine should be so indulgent as to let him and if he is disposed to make them any amendsI shall not be the mean to make us the atonement he thinks our duethe wish is Elizabeth was chiefly struck by his extraordinary deference for Lady Catherineand his kind intention of

In [0]:
response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
key_phrases = response.json()
pprint(key_phrases)